# Recurrent Neural Networks (RNN)
Typical machine learning algorithms for supervised learning assume that the input is independent and identically distributed (IID) data, which means that the training examples are mutually independent and have the same underlying distribution. In this regard, based on the mutual independence assumption, the order in which the training examples are given to the model is irrelevant. For example, if we have a sample consisting of n training examples, x(1), x(2), ..., x(n), the order in which we use the data for training our machine learning algorithm does not matter.

What makes sequences unique, compared to other types of data, is that elements in a sequence appear in a certain order and are not independent of each other, i.e., order matters. Predicting the market value of a particular stock would be an example of this scenario.

## Sequential data vs time series data
Time series data is a special type of sequential data where each example is associated with a dimension for time. In time series data, samples are taken at successive timestamps, and therefore, the time dimension determines the order among the data points. For example, stock prices and voice or speech records are time series data.

On the other hand, not all sequential data has the time dimension. For example, in text data or DNA sequences, the examples are ordered, but text or DNA does not qualify as time series data.

## Representing seqeunces
In the ML context, both features and labels are indexed by time as $(\mathbf{x}^{(1)}, \mathbf{x}^{(2)},\cdots, \mathbf{x}^{(T)})$ and $(y^{(1)}, y^{(2)},\cdots, y^{(T)})$. RNNs are designed for modeling sequences and are capable of remembering past information (they have memory) and processing new events accordingly, which is a clear advantage when working with sequence data. 

## Different categrories of sequence data
If either the input or output is a sequence, the modelin task falls into one of these categors:

(Many-to-one)  Input: sequence, Output: not sequence. Exampel: Classifying whether a person liked a movie based on her comment.

(One-to-many) Input: not sequcne, output: sequence, Example: Image (not sequence) captioning (seqeunce)

(Many-to-many) Both sequence. Example: translation

In an RNN, the hidden layer receives its input from both the input layer of the current time step and the hidden layer from the previous time step. Since, in this case, each recurrent layer must receive a sequence as input, all the recurrent layers except the last one must return a sequence as output (that is, we will later have to set "return_sequences=True"). The behavior of the last recurrent layer depends on the type of problem.

## Computing activations in an RNN
We should note that weight matrices in RNN do not depend on time $t$, i.e., they are shared across time axis. Computing the activation is very much similar to standard multilayers perceptron:
\begin{align}
\mathbf{z}^{(t)}_h &= W_{xh}\mathbf{x}^{(t)} + W_{hh}\mathbf{h}^{(t-1)} + \mathbf{b}_h \\
\mathbf{h}^{(t)} &= \sigma_h(\mathbf{z}^{(t)}_h) 
\end{align}
The activation of th eoutput unit can be computed as
\begin{align}
\mathbf{o}^{(t)} = \sigma_0(W_{ho}h^{(t)}) + \mathbf{b}_o 
\end{align}

The basic idea to derive the gradient of the composition function is similar to before, while a bit more complicated. The overall loss, $L$, is the sum of all loss function at $t=1,\cdots,T$:
\begin{align}
L = \sum_{t=1}^T L^{(t)}
\end{align}
Since the loss at time $t$ depends on the hidden units at all previous times $1:t$, the gradient will be calculated as
\begin{align}
\frac{\partial L^{(t)}}{W_{hh}} = \frac{\partial L^{(t)}}{o^{(t)}} \times \frac{\partial o^{(t)}}{h^{(t)}} \times (\sum_{k=1}^t \frac{\partial h^{(t)}}{h^{(k)}} \times \frac{\partial h^{(k)}}{W_{hh}})
\end{align}
where 
\begin{align}
\frac{\partial h^{(t)}}{h^{(k)}}  = \Pi_{i=k+1}^t \frac{\partial h^{(i)}}{h^{(i-1)}}
\end{align}

## Output-to-hidden and output-to-output recurrence
So far, we saw hidden-to-hidden recurrence. However, we potentially can have output-to-hidden and output-to-output recurrence too (check the corresponding slide). To see how this works in practice, let’s manually compute the forward pass for one of these recurrent types. recurrence. In the following code, we will create a recurrent layer from RNN and perform a forward pass on an input sequence of length 3 to compute the output. We will also manually compute the forward pass and compare the results with those of RNN.


In [1]:
import torch
import torch.nn as nn

torch.manual_seed(1)

rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True) 

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh shape:', w_xh.shape)
print('W_hh shape:', w_hh.shape)
print('b_xh shape:', b_xh.shape)
print('b_hh shape:', b_hh.shape)

W_xh shape: torch.Size([2, 5])
W_hh shape: torch.Size([2, 2])
b_xh shape: torch.Size([2])
b_hh shape: torch.Size([2])


In [2]:
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()

## output of the simple RNN:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))

## manually computing the output:
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))
    print(f'Time step {t} =>')
    print('   Input           :', xt.numpy())
    
    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh    
    print('   Hidden          :', ht.detach().numpy())
    
    if t>0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))

    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.detach().numpy())
    print('   RNN output      :', output[:, t].detach().numpy())
    print()

Time step 0 =>
   Input           : [[1. 1. 1. 1. 1.]]
   Hidden          : [[-0.4701929  0.5863904]]
   Output (manual) : [[-0.3519801   0.52525216]]
   RNN output      : [[-0.3519801   0.52525216]]

Time step 1 =>
   Input           : [[2. 2. 2. 2. 2.]]
   Hidden          : [[-0.88883156  1.2364397 ]]
   Output (manual) : [[-0.68424344  0.76074266]]
   RNN output      : [[-0.68424344  0.76074266]]

Time step 2 =>
   Input           : [[3. 3. 3. 3. 3.]]
   Hidden          : [[-1.3074701  1.886489 ]]
   Output (manual) : [[-0.8649416   0.90466356]]
   RNN output      : [[-0.8649416   0.90466356]]



## The challenges of learning long-range interactions
Back propagation through time introduces some new challenges. Because of the multiplicative factor, in computing the gradients of a loss function, the so-called "vanishing and exploding gradient" problems arise. Basically, $\partial h^{(t)}/\partial h^{(k)}$ has $t-k$ multiplications; therefore, multiplying the weight, $W$, by itself $t – k$ times. As a result, if $|W| < 1$, this factor becomes very small when $t – k$ is large. On the other hand, if the weight of the recurrent edge is $|W| > 1$, then $W^{t–k}$ becomes very large when t – k is large. Note that a large t – k refers to long-range dependencies.

Solutions: 1. Gradient clipping, 2. Truncated backpropagation through time, 3. "LSTM"

